In [1]:
from ROOT import (TCanvas, TPad, TFile, TPaveLabel, 
                  TPaveText, gROOT, TH1F, TH1D, TLegend, 
                  gStyle, TH2F, TChain, TGraphErrors, TText, gPad, gROOT, TTree)
from array import array
import datetime
import numpy as np

np.random.seed(42)
        
from keras.models import Sequential, Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate, SimpleRNN, GRU, Masking, Lambda, Reshape, Dropout, RNN
from keras.optimizers import Adagrad, SGD, RMSprop, Adam

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, LSTM, Conv1D, SimpleRNN, Concatenate
from keras import optimizers
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score
import keras.backend as K
import pickle
#import seaborn as sns

TODAY = datetime.datetime.now().strftime('%Y-%m-%d')

def init_dirs():
    import datetime, os
    for _dir in ["plots", "models"]:
        today_dir = os.path.join(_dir, TODAY)
        if not os.path.isdir(today_dir):
            os.makedirs(today_dir)
            
#init_dirs()
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()


Welcome to JupyROOT 6.14/06


Using TensorFlow backend.


In [2]:
# all methods here
def clean_arrays(data, weights=None):
    new = []
    for d in data:
        #if all(di == 0. for di in d):
        if d[-1]==0.:
            continue
        new.append(d)
    out = np.array(new)
    print("before", data.shape, 'after', out.shape)
    return out

def getweight(wt, Xsec):
    #totalWt = sum(wt)
    wt = Xsec*139*wt
    return wt

def build_array(fname, features, cuts, mll_idx = 10, should_clean_arrays=True, **kwargs):
    #print(features[mll_idx])
    f = TFile.Open(fname)
    tree = f.tree_NOMINAL
    n_events = tree.GetEntries()
    data = np.zeros((n_events, len(features)))
    for n, (event, element) in enumerate(zip(tree, data)):
        for i, feature in enumerate(features):
            try:
                val = getattr(event, feature)
            except:
                val = None
            if feature in cuts and eval("{}{}".format(val, cuts[feature])):
                element[i] = getattr(event, feature)
            elif feature in cuts:
                element = np.zeros(len(features))
                break
            elif feature == 'train_mass':
                element[i] = getattr(event, features[mll_idx])
            elif feature == 'MZ1_MZ2':
                element[i] = getattr(event, 'MZ1')-getattr(event, 'MZ2')
            else:
                element[i] = val
    return clean_arrays(data) if should_clean_arrays else data

features = ('train_mass', ## 0
            'PtL1',       ## 1
            'PtL2',       ## 2
            'PtL3',       ## 3
            'PtL4',       ## 4
            'EtaL1',      ## 5
            'EtaL2',      ## 6
            'EtaL3',      ## 7
            'EtaL4',      ## 8
            'MZ1',        ## 9
            'MZ2',        ## 10
            'MZ1_MZ2',    ## 11
            'PtZ1',       ## 12
            'PtZ2',       ## 13
            'MZZ',        ## 14
            'PtZZ',       ## 15
            'DeltaRl12',  ## 16
            'DeltaRl34',  ## 17
            'dEtal12',    ## 18
            'dEtal34',    ## 19
            'run',        ## 20
            'event',      ## 21
            'quadtype',   ## 22
            'weight')     ## 23

# these are your cuts that preselect events, empty now
#cuts = dict()
#cuts = {"MZZ": "<120"}
cuts = {"quadtype": "==2",
        "MZZ": "<180"}

def unison_shuffled_copies(*arr):
    assert all(len(a) for a in arr)
    p = np.random.permutation(len(arr[0]))
    return (a[p] for a in arr)

mll_idx = 9 # 10 when signal sample < 40GeV; 9 when signal >40GeV

print("have {} features".format(len(features)))
print("mZp is", features[mll_idx])

have 24 features
mZp is MZ1


# Loop over samples and create skimmed files only containing interesting features

Make sure all directories are correct. In this case I save my train and test data to ``example_hep/train`` and ``example_hep/test``. I only use the files in ``train`` for training and for evaluation I only use ``test``.

The msamp list contains the sample number and masses.

## So first for the signal samples

In [3]:
import io
import numpy as np

msamp = [42,45,48,51,54,57,60,63,66,69,72,75]

## Need to do rescale to only training datasets
## firstly sum them together to fit transform
#raw_bkg1 = build_array(
#    '/Users/zhuheling/ML/keras/example_hep/myTest/data/tree_bk.root',
#    features, cuts, mll_idx=10)
#raw_train = raw_bkg1[0:int(len(raw_bkg1)*0.6)]
#raw_train[:,0] = 40
#print (np.amax(raw_train, axis=0), np.amin(raw_train, axis=0))
#for mass in msamp:
#    if mass<40:
#        m_idx=10
#    else:
#        m_idx=9
#    raw_signal_add = build_array(
#        '/Users/zhuheling/ML/keras/example_hep/myTest/data/tree_{}GeV.root'.format(mass), 
#        features, cuts, m_idx)
#    raw_train_add = raw_signal_add[0:int(len(raw_signal_add)*0.6)]
#    raw_train = np.concatenate((raw_train,raw_train_add))
#print (np.amax(raw_train, axis=0), np.amin(raw_train, axis=0))

#min_max_scaler.fit_transform(raw_train[:,0:20])

for mass in msamp:
    if mass<40:
        m_idx=10
    else:
        m_idx=9
    signal = build_array(
        '/Users/zhuheling/ML/keras/example_hep/myTest/data/tree_{}GeV.root'.format(mass), 
        features, cuts, m_idx)
    #signal = np.zeros( (len(raw_signal), 43))
    #signal[:,0:20] = min_max_scaler.transform(raw_signal[:,0:20])
    #signal[:,20:40] = raw_signal[:,0:20]
    #signal[:,40:43] = raw_signal[:,20:23]
    #print(raw_signal[0:3])
    #print(signal[0:3])
   
    with io.open('/Users/zhuheling/ML/keras/example_hep/myTest/Below180/UnscaledData/data_npy/tree_{}GeV.npy'.format(mass), 'wb') as f:
        np.save(f, signal)

    n_trains = int(len(signal)*0.6)
    n_vals = int(len(signal)*0.8)
    
    strain = signal[0:n_trains]
    with io.open('/Users/zhuheling/ML/keras/example_hep/myTest/Below180/UnscaledData/train/tree_{}GeV.npy'.format(mass), 'wb') as f:
        np.save(f, strain)
    
    sval = signal[n_trains:n_vals]
    with io.open('/Users/zhuheling/ML/keras/example_hep/myTest/Below180/UnscaledData/Validation/tree_{}GeV.npy'.format(mass), 'wb') as f:
        np.save(f, sval)
    
    stest = signal[n_vals:]
    with io.open('/Users/zhuheling/ML/keras/example_hep/myTest/Below180/UnscaledData/test/tree_{}GeV.npy'.format(mass), 'wb') as f:
        np.save(f, stest)
        
    print ("Training, validation, testing: ", strain.shape, sval.shape, stest.shape)


before (4493, 24) after (4490, 24)
Training, validation, testing:  (2694, 24) (898, 24) (898, 24)
before (4496, 24) after (4493, 24)
Training, validation, testing:  (2695, 24) (899, 24) (899, 24)
before (4256, 24) after (4254, 24)
Training, validation, testing:  (2552, 24) (851, 24) (851, 24)
before (4175, 24) after (4172, 24)
Training, validation, testing:  (2503, 24) (834, 24) (835, 24)
before (3800, 24) after (3797, 24)
Training, validation, testing:  (2278, 24) (759, 24) (760, 24)
before (6859, 24) after (6851, 24)
Training, validation, testing:  (4110, 24) (1370, 24) (1371, 24)
before (5792, 24) after (5786, 24)
Training, validation, testing:  (3471, 24) (1157, 24) (1158, 24)
before (9519, 24) after (9513, 24)
Training, validation, testing:  (5707, 24) (1903, 24) (1903, 24)
before (7567, 24) after (7561, 24)
Training, validation, testing:  (4536, 24) (1512, 24) (1513, 24)
before (4757, 24) after (4750, 24)
Training, validation, testing:  (2850, 24) (950, 24) (950, 24)
before (4035

# And now for the background samples
Need to re-define the mass parameter (train_mass) for backgrounds used for training.

In [9]:
import io
import numpy as np

bkg1 = build_array(
    '/Users/zhuheling/ML/keras/example_hep/myTest/data/tree_qqZZ.root',
    features, cuts, mll_idx=9)

#print(raw_bkg1[0:3])
#bkg1 = np.zeros( (len(raw_bkg1), 43))
#bkg1[:,0:20] = min_max_scaler.transform(raw_bkg1[:,0:20])
#bkg1[:,20:40] = raw_bkg1[:,0:20]
#bkg1[:,40:43] = raw_bkg1[:,20:23]
#print(bkg1[0:3])

# Xsec: ggZZ: 10.163 fb; qqZZ: 1252 fb; ggZZ_low: 9.958 fb
# Sumofweight: ggZZ: 1176146.3; qqZZ: 5043806; ggZZ_low: 100064.62
#print(bkg1[0:5,-1])
bkg1[:,-1] = getweight(bkg1[:,-1], 1252./5043806.)
#print(bkg1[0:5,-1])
with io.open('/Users/zhuheling/ML/keras/example_hep/myTest/Below180/UnscaledData/data_npy/tree_qqZZ.npy', 'wb') as f:
        np.save(f, bkg1)

n_trains = int(len(bkg1)*0.6)
n_vals = int(len(bkg1)*0.8)

btrain = bkg1[0:n_trains]
with io.open('/Users/zhuheling/ML/keras/example_hep/myTest/Below180/UnscaledData/train/tree_qqZZ.npy', 'wb') as f:
    np.save(f, btrain)
    
bval = bkg1[n_trains:n_vals]
with io.open('/Users/zhuheling/ML/keras/example_hep/myTest/Below180/UnscaledData/Validation/tree_qqZZ.npy', 'wb') as f:
    np.save(f, bval)
    
btest = bkg1[n_vals:]
with io.open('/Users/zhuheling/ML/keras/example_hep/myTest/Below180/UnscaledData/test/tree_qqZZ.npy', 'wb') as f:
    np.save(f, btest)
    
print ("Training, validation, testing: ", btrain.shape, bval.shape, btest.shape)
print ("Weighted training, validation, testing: ", sum(btrain[:,-1]), sum(bval[:,-1]), sum(btest[:,-1]))

before (52953, 24) after (22918, 24)
Training, validation, testing:  (13750, 24) (4584, 24) (4584, 24)
Weighted training, validation, testing:  568.2608611656711 191.02904621943267 189.81990211790256
